In [1]:
import os
import pandas as pd 

In [9]:
dd = '.'
assessments = []
for f in os.listdir(dd):
    df = pd.read_csv(dd+'/'+f, index_col=0)
    assessments.append(df)
    print("Number of relevants per consensus", df[df["consensus"]==True]['doc_id'].count(), "out of", len(df))
    print("Number of relevants per majority", df[df["majority"]==True]['doc_id'].count(), "out of", len(df))
    print()

Number of relevants per consensus 3 out of 103
Number of relevants per majority 7 out of 103

Number of relevants per consensus 3 out of 109
Number of relevants per majority 8 out of 109

Number of relevants per consensus 11 out of 103
Number of relevants per majority 24 out of 103

Number of relevants per consensus 13 out of 134
Number of relevants per majority 27 out of 134

Number of relevants per consensus 3 out of 117
Number of relevants per majority 11 out of 117

Number of relevants per consensus 14 out of 114
Number of relevants per majority 23 out of 114

Number of relevants per consensus 3 out of 121
Number of relevants per majority 12 out of 121

Number of relevants per consensus 8 out of 91
Number of relevants per majority 16 out of 91

Number of relevants per consensus 8 out of 107
Number of relevants per majority 16 out of 107

     doc_id                                               text  \
0    405250  No creo que le hecho de dejar de jugar sea jod...   
1     75525  E

In [11]:

final = pd.concat(assessments)
print(final.columns)

Index(['doc_id', 'text', 'manu.couto1k_rels', 'marcosfp_rels',
       'ezra.mario_rels', 'consensus', 'majority'],
      dtype='object')


In [20]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(final['manu.couto1k_rels'].values, final['marcosfp_rels'].values)

0.4621155767146289

In [14]:
cohen_kappa_score(final['manu.couto1k_rels'], final['ezra.mario_rels'])

0.41305269904706243

In [15]:
cohen_kappa_score(final['marcosfp_rels'], final['ezra.mario_rels'])

0.44795325705143807

In [28]:
import krippendorff
krippendorff.alpha(reliability_data=[list(final['marcosfp_rels'].astype(int).values), list(final['ezra.mario_rels'].astype(int).values),list(final['manu.couto1k_rels'].astype(int).values)])

0.43685967429961936

In [2]:
dd = './gambling_asesores_primero/'
assessments = []
for f in os.listdir(dd):
    df = pd.read_csv(dd+'/'+f, index_col=0)
    assessments.append(df)
    print("Number of relevants per consensus", df[df["consensus"]==True]['doc_id'].count(), "out of", len(df))
    print("Number of relevants per majority", df[df["majority"]==True]['doc_id'].count(), "out of", len(df))
    print()

Number of relevants per consensus 4 out of 77
Number of relevants per majority 15 out of 77

Number of relevants per consensus 2 out of 74
Number of relevants per majority 10 out of 74

Number of relevants per consensus 7 out of 62
Number of relevants per majority 16 out of 62

Number of relevants per consensus 0 out of 71
Number of relevants per majority 5 out of 71

Number of relevants per consensus 2 out of 60
Number of relevants per majority 8 out of 60

Number of relevants per consensus 2 out of 63
Number of relevants per majority 9 out of 63

Number of relevants per consensus 11 out of 72
Number of relevants per majority 13 out of 72

Number of relevants per consensus 4 out of 67
Number of relevants per majority 9 out of 67

Number of relevants per consensus 3 out of 67
Number of relevants per majority 7 out of 67



In [3]:
final = pd.concat(assessments)
print(final.columns)

Index(['doc_id', 'text', 'manu.couto1k_rels', 'manu.couto1k_posi',
       'marcosfp_rels', 'marcosfp_posi', 'ezra_rels', 'ezra_posi', 'consensus',
       'majority', 'consesus_doub', 'majority_doub', 'posible_consensus_1',
       'posible_consensus_2', 'Unnamed: 15',
       '¿Ha mentido a su familia o a otras personas para ocultar el alcance de su ludopatía?',
       '¿Has jugado para escapar de los problemas o cuando te sientes se siente deprimido, ansioso o mal',
       '¿Se ha encontrado a menudo pensando en el juego (por ejemplo, reviviendo experiencias de juego',
       '¿Ha pedido ayuda a otras personas para que le proporcionen el dinero necesario para aliviar una',
       '¿Ha intentado repetidamente sin éxito controlar, reducir o dejar de jugar?',
       '¿Se ha vuelto inquieto o irritable al intentar reducir o dejar de jugar?',
       'Después de perder dinero apostando, ¿has vuelto otro día para para desquitarse?',
       '¿Ha necesitado apostar cada vez más dinero para conse